#### VAR Automation

#### Description:
###### This file contains the code for VAR automation, along with csv's for intermediate results such as portfolio holdings, price changes

###### Author: Nishchal Gaba
###### Created on: January 12, 2022

In [1]:
# Import packages
import os
import time
import sys
import random
from datetime import datetime, timedelta

import json
import requests
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

#### Utility Methods

In [2]:
# Used for Coin Gecko
# token_names = {"DAI"  : 'dai',
#             "GUSD"  : 'gemini-dollar',
#             # "SUSD"  : 'susd',
#             "TUSD"  : 'true-usd',
               
               
#             "USDC"  : 'usd-coin',
#             # "USDP"  : 'usdp',
#             "USDT"  : "tether",
#             "BAL"   : "balancer",
#             "WETH"   : "ethereum",
#             "LINK"  : "chainlink",
#             "MKR"   : "maker",
#             "RAI"   : "rai",
#             "UNI"   : "uniswap",
#             "WBTC"  : "wrapped-bitcoin",
#             "XSUSHI": "xsushi",
#             "YFI"   : "yearn-finance",
#             "BUSD"  : "binance-usd",
#             "FEI"   : "fei-usd",
#             "FRAX"  : "frax",
#             "AAVE"  : "aave",
#             "AMPL"  : "ampleforth",
#             "BAT"   : "basic-attention-token",
#             "CRV"   : "curve-dao-token",
#             "DPI"   : "defipulse-index",
#             # # "ENJ"   : "enjin-coin",
#             "KNC"   : "kyber-network-crystal",
#             "MANA"  : "decentraland",
#             # "REN"   : "ren",
#             "RENFIL": "renfil",
#             # "SNX"   :"synthetix-network-token",
#             "ZRX"   : "0x"
#             }

In [3]:
# Coingecko data fetcher
def getCoinGeckoHistoricalData(_token):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _token(str): Token Symbol (actual token will be fetched from token_names dictionary)
            
        Returns::
            _resDf(pandas dataframe): Returns the dates (yyyy-mm-dd) and prices (USD) in a dataframe
    '''
    try:
        # Making the coingecko historical API request
        coinGeckoTokenCall = token_names[_token]
        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(coinGeckoTokenCall)+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        
        # Converting the result to JSON
        resJSON = request.json()
        
        # Creating empty dataframe
        resDf = pd.DataFrame(resJSON['prices'], columns = ["timestamp", "price"])
        
        # We divide by 1000 to convert the default milliseconds from CoinGecko to seconds
        # Get the human readable date from timestamp
        resDf['timestamp'] = resDf['timestamp'].apply(lambda x: str(datetime.fromtimestamp(x/1000).date()))
        
        # Return the resultant dataframe
        return resDf
        
        
    except Exception as e:
        print("Exception occured while fetching Coingecko Data")
        return str(e)
    

In [4]:
# AAVE V2 data for a particular date
def aaveV2HistoricalData(_date):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _date(str): date for which data is needed in (%m-%d-%Y) format
            
        Returns::
            _resDf(pandas dataframe): Token, CurrentPrice, availableLiquidity, decimals,
                                        totalLiquidity, utilizationRate, Assets
    '''
    
    try:
        # Starting Index
        ind = 0
        
        # Calling the Aave V2
        aaveV2res = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
        resDf = pd.DataFrame(columns=['Token', 'CurrentPrice', 'availableLiquidity','decimals', 'totalLiquidity','utilizationRate','Assets'])
        
        # Iterating through tokens in AaveV2 API
        for token in aaveV2res:
            try:
                resDf.loc[ind] = [token['symbol']] + [float(token['referenceItem']['priceInUsd'])] + \
                                [float(token['availableLiquidity'])] + [float(token['decimals'])]+ \
                                [float(token['totalLiquidity'])]+ [float(token['referenceItem']['utilizationRate'])] + \
                                [float(token['totalDebt'])]

                ind += 1

            except Exception as e:
                print("**", token, e) #str(asset), str(price_mean), str(price_std), str(cap_mean),str(cap_std), str(vol_mean) , str(vol_std))  
    #             errors.append(token)
                # transactionResponse["error"] = True
                print("Exception occured while fetching Aave V2 Token wise Data")
                return str(e)


        # totalLiquidity == Market
        resDf['totalLiquidity']       = resDf['totalLiquidity'] * resDf['CurrentPrice']
        resDf['Assets']               = resDf['Assets'] * resDf['CurrentPrice']
        resDf['Position']             = resDf['Assets'] - resDf['totalLiquidity']
        resDf['Position($,Billion)']  = resDf['Position'] / pow(10, 9)


        # Return the resultant dataframe
        return resDf
        
    except Exception as e:
        print("Exception occured while fetching Aave V2 API Data")
        return str(e)


In [5]:
# def change():
#     query = '''
#                 {
#                 reserves
#                 (where:{
#                     symbol:"WBTC"
#                     })
#                 {
#                     id
#                     symbol
#                     name
#                     usageAsCollateralEnabled
#                         borrowingEnabled
#                     baseLTVasCollateral
#                     totalDeposits
#                     totalLiquidity
#                     totalATokenSupply
#                     reserveFactor
                    
#                 }
#                 }
#             '''

#     V2_sample_transport       = RequestsHTTPTransport(
#         url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
#         verify=True,
#         retries=5)
#     V2_client                 = Client(transport=V2_sample_transport)
#     response                  = V2_client.execute(gql(query))

#     # Coingecko Mapping for each token name to fetch historical prices
#     names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         "REN"   : "ren",
#         "RENFIL": "renfil",
#         "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }
    
    
#     columns = []
#     columns.append('timestamp')
#     for key in list(names.keys()):
#         columns.append(key+'_price')
#         columns.append(key+'_%change(10)')


#     df        = pd.DataFrame(columns = columns)
#     LOOK_BACK = 375
#     LIMIT     = 5
#     WBTC      = []

#     request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#     data      = request.json()
#     WBTC      = data['prices']
#     tt        = []
#     P         = []
#     for day in data['prices'][-LOOK_BACK:]:
#         tt.append(day[0]/1000)
#         P.append(day[1])
#     df['WBTC_price'] = P
#     df['timestamp']  = tt

#     track = {}
#     for asset in list(names.keys()):#[:LIMIT]:
#         if asset == 'WBTC':
#             continue
#         request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#         data = request.json()
#         tt = []
#         P = []
        
#         if len(data['prices']) < LOOK_BACK:
#             req = LOOK_BACK - len(data['prices'])
#             addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
#             data['prices'] = addition + data['prices']
#             track[asset] = req
#         else:
#             data['prices'] = data['prices'][-LOOK_BACK:]
        
#         for day in data['prices']:
#             tt.append(day[0]/1000)
#             P.append(day[1])
#         df[asset+'_price'] = P
#         df['timestamp']  = tt

#     df['timestamp'] = tt

#     for asset in list(names.keys()):#[:LIMIT]:
        
#         diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
#         change = []
#         for i in range(10):
#             change.append('Nan')
#         for i in range(10,df.shape[0]):
#             change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#         df[asset+'_%change(10)'] = change
        
#         diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
#         change = []
#         for i in range(10):
#             change.append('Nan')
#         for i in range(10,df.shape[0]):
#             change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#         df[asset+'_%change(1)'] = change
        
#     for key in track.keys():
#         df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

#     df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
#     df  = df.iloc[::-1]
#     df  = df.reset_index()
#     df = df.drop(['index'], axis=1)

        
#     return df

In [6]:
# Coingecko Mapping for each token name to fetch historical prices
names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        "REN"   : "ren",
        "RENFIL": "renfil",
        "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }

In [7]:
# Funtion to give historical price changes for each token mentioned in Names Dictionary
def change():
    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        P    = []
        if request.status_code != 200:
            data = {}
            data['prices'] = WBTC[ -LOOK_BACK: ]
            for day in data['prices']:
                P.append(day)
        else:   
            data = request.json()
            tt   = []
            

            if len(data['prices']) < LOOK_BACK:
                req = LOOK_BACK - len(data['prices'])
                addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
                data['prices'] = addition + data['prices']
                track[asset] = req
            else:
                data['prices'] = data['prices'][-LOOK_BACK:]

            for day in data['prices']:
                tt.append(day[0]/1000)
                P.append(day[1])
                
        df[asset+'_price'] = P
        df['timestamp']  = tt
    df['timestamp'] = tt
    
    print("PRICES FETECHED")
    
    unknown_assets = ['PAX']
    for asset in list(names.keys()):#[:LIMIT]:
        try:
            change = []
            diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
            
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(10)'] = change

            diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
            change = []
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(1)'] = change  
            
        except Exception as e:
            unknown_assets.append(asset)
            print("Error ", asset, e)
    
    for asset in unknown_assets:
        df[asset+'_%change(1)']  = df['WBTC_%change(1)'] 
        df[asset+'_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(1)'] = df['WBTC_%change(1)'] 
        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)

    df = df[:-10]

    print("CHECK POINT ")
    return df
change_df = change()

PRICES FETECHED
Error  SUSD unsupported operand type(s) for -: 'list' and 'list'
Error  ENJ unsupported operand type(s) for -: 'list' and 'list'
Error  REN unsupported operand type(s) for -: 'list' and 'list'
Error  SNX unsupported operand type(s) for -: 'list' and 'list'
CHECK POINT 


/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#### Getting the AAVEV2 assets via Subgraph GQL


In [8]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [9]:
# query = '''
#                 {
#                 reserves
#                 (where:{
#                     symbol:"WBTC"
#                     })
#                 {
#                     id
#                     symbol
#                     name
#                     usageAsCollateralEnabled
#                         borrowingEnabled
#                     baseLTVasCollateral
#                     totalDeposits
#                     totalLiquidity
#                     totalATokenSupply
#                     reserveFactor
                    
#                 }
#                 }
#             '''

In [10]:
# V2_sample_transport       = RequestsHTTPTransport(
#         url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
#         verify=True,
#         retries=5)

In [11]:
# V2_client                 = Client(transport=V2_sample_transport)
# V2_sample_response                  = V2_client.execute(gql(query))

In [12]:
# V2_sample_response

In [13]:
# Required Date
# By default set to current date
# requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate = '12-12-2021' #"01-10-2022" #mm-dd-yyyy

In [14]:
def VARDate(_date):
    '''
        Gets the VAR results, AaveV2 for a particular position and price change dataframe
        
        Args::
            _date(str): Date in "%m-%d-%Y" format
        
        Returns::
            aaveV2Res(pandas dataframe): AaveV2 Historical Data for the given date
            df (pandas dataframe): Price change dataframe
    '''
    # Aave V2 Data for particular date
    aaveV2Res = aaveV2HistoricalData(_date)
    
    # Price Change Dataframe
    df = change()
    
    # Flipping the date format
    reqDate = datetime.strptime(_date, '%m-%d-%Y').strftime('%Y-%m-%d')
    
    # Converting str to datetime
    df['date']= pd.to_datetime(df['date'])
    # Remove dates above the required date
    mask = (df['date'] <= reqDate)
    df = df.loc[mask]
    
    # Converting the date column back to str
    df['date']=df['date'].astype(str)
    
    REQUIRED_1 = []
    REQUIRED_10 = []
    historicalVAR = pd.DataFrame(columns=['date', 'VAR_10', 'VAR_1'])
    hist_ind = 0
    for required_row in df.iterrows():
        required_row = required_row[1]
        SUM_10 = []
        SUM_1 = []
            
        for row in aaveV2Res.iterrows():
            try:
                SUM_10.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(10)'])/100))
                SUM_1.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(1)'])/100))
            except Exception as e:
                print("ERROR", e)
                pass

        REQUIRED_1.append(sum(SUM_1))
        REQUIRED_10.append(sum(SUM_10))

        historicalVAR.loc[hist_ind] = [required_row['date']] + [sum(SUM_10)] + [sum(SUM_1)]
        hist_ind += 1 
        
    # Historical VAR for past 362 days    
    historicalVAR = historicalVAR[:365]
    
    
    
    REQUIRED_10 = historicalVAR.sort_values('VAR_10').iloc[3]
    REQUIRED_1 = historicalVAR.sort_values('VAR_1').iloc[3]

    REQUIRED_10_95 = historicalVAR.sort_values('VAR_10').iloc[9]
    REQUIRED_1_95 = historicalVAR.sort_values('VAR_1').iloc[9]

    var_date_10_99p = REQUIRED_10['date']
    var_date_10_95p = REQUIRED_10_95['date']
    var_date_1_99p = REQUIRED_1['date']
    var_date_1_95p = REQUIRED_1_95['date']
    
#     print(historicalVAR)
    
    testdf = pd.DataFrame(columns=['SYMBOL', 'totalLiquidity', 'totalDebt'])
    ind = 0
    
    AAVE_API_RESULT  = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
    
    for i in AAVE_API_RESULT:
        testdf.loc[ind] = [i['symbol']] + [float(i['totalLiquidity']) * float(i['referenceItem']['priceInUsd']) ]\
                    + [float(i['totalDebt']) * float(i['referenceItem']['priceInUsd'] )] #+ [i['referenceItem']['priceInUsd'] * ]
        ind += 1

    TOTAL_LIABILITIES = aaveV2Res['totalLiquidity'].sum()
    TOTAL_ASSETS      = aaveV2Res['Assets'].sum()
    VAR_10_DAY_99_P = str(REQUIRED_10['VAR_10'])
    VAR_DATE_10_DAY_99_P = var_date_10_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['10_day_95p'] = str(REQUIRED_10_95['VAR_10'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_10_day_95p'] = var_date_10_95p

#     # transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1[5])
#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_99p'] = var_date_1_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_95p'] = str(REQUIRED_1_95['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_95p'] = var_date_1_95p

    RELATIVE_VAR_ASSETS = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_ASSETS)
    RELATIVE_VAR_LIABILITIES = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_LIABILITIES)
    #TOTAL_ASSETS
    #TOTAL_LIABILITIES
    return aaveV2Res, df, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P

In [15]:
# aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)
# Date in %m-%d-%Y format
aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)

PRICES FETECHED
Error  SUSD unsupported operand type(s) for -: 'list' and 'list'
Error  ENJ unsupported operand type(s) for -: 'list' and 'list'
Error  REN unsupported operand type(s) for -: 'list' and 'list'
Error  SNX unsupported operand type(s) for -: 'list' and 'list'


/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CHECK POINT 


In [16]:
len(priceChangeDf.columns)

97

In [24]:
aaveV2Res

,Token,CurrentPrice,availableLiquidity,decimals,totalLiquidity,utilizationRate,Assets,Position,"Position($,Billion)"
0,TUSD,1.004101,4.909780e+07,18.0,1.164821e+08,0.634753,6.718296e+07,-4.929913e+07,-0.049299
1,RAI,3.015978,2.068256e+06,18.0,3.125559e+07,0.552307,2.501778e+07,-6.237813e+06,-0.006238
2,GUSD,1.000000,2.911591e+06,2.0,1.439318e+07,0.769164,1.148158e+07,-2.911591e+06,-0.002912
3,YFI,21868.937652,1.469237e+03,18.0,3.442117e+07,0.041959,2.290508e+06,-3.213066e+07,-0.032131
4,BAT,1.124654,4.199938e+06,18.0,5.174775e+06,0.082713,4.512972e+05,-4.723478e+06,-0.004723
5,MANA,3.655208,1.337645e+07,18.0,5.045326e+07,0.024376,1.559556e+06,-4.889371e+07,-0.048894
6,DPI,254.805830,1.789095e+05,18.0,4.583553e+07,0.005982,2.483377e+05,-4.558719e+07,-0.045587
7,UNI,15.694045,2.650525e+06,18.0,4.203471e+07,0.258357,4.372475e+05,-4.159746e+07,-0.041597
8,WBTC,50252.278723,2.647879e+04,8.0,1.361395e+09,0.028352,3.077558e+07,-1.330619e+09,-1.330619
9,REN,0.571570,4.418143e+07,18.0,2.586895e+07,0.023726,6.161660e+05,-2.525278e+07,-0.025253


In [17]:
VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P

('-4.645916248358751', '2021-01-06')

In [18]:
VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P

('-4.645916248358751', '2021-01-06')

In [19]:
TOTAL_ASSETS, TOTAL_LIABILITIES

(5770349166.414649, 14640683379.854826)

In [20]:
priceChangeDf.head()

,timestamp,DAI_price,DAI_%change(10),GUSD_price,GUSD_%change(10),SUSD_price,SUSD_%change(10),TUSD_price,TUSD_%change(10),USDC_price,...,MANA_%change(1),RENFIL_%change(1),ZRX_%change(1),PAX_%change(1),PAX_%change(10),SUSD_%change(1),ENJ_%change(1),REN_%change(1),SNX_%change(1),date
32,1.639267e+09,0.999549,-0.176785,0.996231,-0.404317,"[1639267200000, 49270.20633816324]",-13.8825,1.000780,0.0949368,0.997664,...,5.17896,2.01589,3.54943,3.50354,-13.8825,3.50354,3.50354,3.50354,3.50354,2021-12-12
33,1.639181e+09,0.995229,-0.277911,0.992990,-0.461191,"[1639180800000, 47265.73410567076]",-17.2476,0.994981,-0.269506,0.996124,...,-5.21831,0.578051,-5.00301,-0.91198,-17.2476,-0.91198,-0.91198,-0.91198,-0.91198,2021-12-11
34,1.639094e+09,0.998008,-0.0602507,0.994984,-0.316327,"[1639094400000, 47786.630321451]",-17.4195,0.995632,-0.41104,0.998575,...,-5.66386,-3.24743,-3.83371,-4.77216,-17.4195,-4.77216,-4.77216,-4.77216,-4.77216,2021-12-10
35,1.639008e+09,1.001356,-0.0281601,0.997215,0.3366,"[1639008000000, 50548.12397229681]",-11.7688,0.999487,0.0632515,1.000100,...,-1.13591,0.731351,1.09974,-0.0489255,-11.7688,-0.0489255,-0.0489255,-0.0489255,-0.0489255,2021-12-09
36,1.638922e+09,1.002961,0.416771,0.996415,0.326707,"[1638921600000, 50576.15367864407]",-7.15412,1.000313,0.0920468,1.002005,...,0.75511,2.70829,3.08074,0.105883,-7.15412,0.105883,0.105883,0.105883,0.105883,2021-12-08


In [21]:
# Position on a particular
aaveV2Res

,Token,CurrentPrice,availableLiquidity,decimals,totalLiquidity,utilizationRate,Assets,Position,"Position($,Billion)"
0,TUSD,1.004101,4.909780e+07,18.0,1.164821e+08,0.634753,6.718296e+07,-4.929913e+07,-0.049299
1,RAI,3.015978,2.068256e+06,18.0,3.125559e+07,0.552307,2.501778e+07,-6.237813e+06,-0.006238
2,GUSD,1.000000,2.911591e+06,2.0,1.439318e+07,0.769164,1.148158e+07,-2.911591e+06,-0.002912
3,YFI,21868.937652,1.469237e+03,18.0,3.442117e+07,0.041959,2.290508e+06,-3.213066e+07,-0.032131
4,BAT,1.124654,4.199938e+06,18.0,5.174775e+06,0.082713,4.512972e+05,-4.723478e+06,-0.004723
5,MANA,3.655208,1.337645e+07,18.0,5.045326e+07,0.024376,1.559556e+06,-4.889371e+07,-0.048894
6,DPI,254.805830,1.789095e+05,18.0,4.583553e+07,0.005982,2.483377e+05,-4.558719e+07,-0.045587
7,UNI,15.694045,2.650525e+06,18.0,4.203471e+07,0.258357,4.372475e+05,-4.159746e+07,-0.041597
8,WBTC,50252.278723,2.647879e+04,8.0,1.361395e+09,0.028352,3.077558e+07,-1.330619e+09,-1.330619
9,REN,0.571570,4.418143e+07,18.0,2.586895e+07,0.023726,6.161660e+05,-2.525278e+07,-0.025253


In [22]:
# Test call for Aave V2 for current date
# This is also used for Position data for a particular date
# testRest = aaveV2HistoricalData(requiredDate)

In [23]:
# testRest